In [ ]:
import os
import json

import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression

from utils.func import read_data, softmax
from utils.metric import evaluate

In [ ]:
model_name = "LLaVA-7B"
prompt = "oe"

In [ ]:
_, x_train, y_train = read_data(model_name, "VizWiz", split="train", prompt=prompt, token_idx=0)
val_data, x_val, y_val = read_data(model_name, "VizWiz", split="val", prompt=prompt, token_idx=0, return_data=True)

print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

In [ ]:
if prompt == "oe":
    label_list = [ins['label'] for ins in val_data]
    val_labeled = json.load(open(f'./output/{model_name}/VizWiz_val_oe_labeled.json'))
    llava_pred = [0 if ins["is_answer"] == 'no' else 1 for ins in val_labeled]
    evaluate(label_list, llava_pred, show=True)

### Direrctly check the responses

In [ ]:
label_list = [ins['label'] for ins in val_data]
pred_list = [0 if "unanswerable" in ins['response'].lower() else 1 for ins in val_data]
evaluate(label_list, pred_list, show=True)

### Use the probability of a spcific token

In [ ]:
label_list = [ins['label'] for ins in val_data]
pred_list = 1 - softmax(x_val)[:, 853]
evaluate(label_list, pred_list, show=True)

### Linear probing

In [ ]:
model = LogisticRegression()
model.fit(x_train, y_train)

y_pred = model.predict_proba(x_val)[:, 1]
_, _, _, auroc = evaluate(y_val, y_pred, show=True)